In [10]:
import itertools
import re
import json
import random
import time

In [4]:
class CNF:
    def __init__(self, dimacs):
        dimacs_tokens = re.split('\s+', dimacs)
        self.num_of_vars = int(dimacs_tokens[2])
        self.num_of_clauses = int(dimacs_tokens[3])
        clauses_dimacs = [int(x) for x in dimacs_tokens[4:]]
        self.clauses = [list(clause) for is_zero, clause 
                    in itertools.groupby(clauses_dimacs, lambda x: x == 0) 
                    if not is_zero]
        
    def evaluate(self, valuation):
        for clause in self.clauses:
            clause_sat = False
            for literal in clause:
                val_idx = abs(literal) - 1
                if literal > 0 and valuation[val_idx] or literal < 0 and not valuation[val_idx]:
                    clause_sat = True
                    break
            if not clause_sat:
                return False
        return True

In [5]:
class Problem:
    def __init__(self, file_name):
        try:
            with open(file_name, 'r') as f:
                dimacs_list = json.load(f)
                self.cnf_list = [CNF(dimacs) for dimacs in dimacs_list]
                self.num_of_vars = self.cnf_list[0].num_of_vars
        except IOError:
            print(f'Error opening file {file_name}')
            exit(1)

In [29]:
class Solution:
    def __init__(self, problem, code=None):
        self.problem = problem
        if code is None:
            self.code = random.choices([True, False], k=self.problem.num_of_vars)
        else:
            self.code = code
        self.fitness = self.calc_fitness()
        
    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def calc_fitness(self):
        num_of_sat = 0
        for formula in self.problem.cnf_list:
            if formula.evaluate(self.code):
                num_of_sat += 1
        return 1 / (num_of_sat + 1)
    
    def get_num_sat(self):
        return round(1 / self.fitness - 1)

In [65]:
class BFSolver:
    @staticmethod
    def solve(problem):
        solutions = [Solution(problem, valuation) for valuation in 
                     itertools.product([True, False], repeat=problem.num_of_vars)]
        return max(solutions)

In [77]:
class GeneticSolver:
    def __init__(self, num_of_generations=50, population_size=10, mutation_probability=0.2):
        self.num_of_generations = num_of_generations
        self.population_size = population_size
        self.mutation_probability = mutation_probability

    def _generate_population(self, problem):
        return [Solution(problem) for _ in range(self.population_size)]

    def _selection(self, population):
        # FIX maybe better _selection, or lower mutation
        return random.choices(population, weights=[s.fitness for s in population], k=1)[0]

    def _crossover(self, problem, parent_1, parent_2):
        bp = random.randrange(len(parent_1.code))
        child_1_code = parent_1.code[:bp] + parent_2.code[bp:]
        child_2_code = parent_2.code[:bp] + parent_1.code[bp:]
        return Solution(problem, child_1_code), Solution(problem, child_2_code)

    def _mutate(self, chromosome):
        if random.random() < self.mutation_probability:
            index = random.randrange(len(chromosome.code))
            chromosome.code[index] = not chromosome.code[index]
            chromosome.fitness = chromosome.calc_fitness()

    def solve(self, problem):
        population = self._generate_population(problem)
        new_population = [None for _ in range(len(population))]
        best_solution = max(population)
        for i in range(self.num_of_generations):
            for j in range(self.population_size // 2):
                parent_1 = self._selection(population)
                parent_2 = self._selection(population)
                child_1, child_2 = self._crossover(problem, parent_1, parent_2)
                self._mutate(child_1)
                self._mutate(child_2)
                new_population[2 * j] = child_1
                new_population[2 * j + 1] = child_2
            population = new_population
            best_solution = max(best_solution, max(population))
            print(f'generation[{i}] :  {best_solution.get_num_sat()}')
        return best_solution

In [80]:
p = Problem('problem_instances/1.json')

result_bf = BFSolver.solve(p)
print(f'{result_bf.code} {result_bf.get_num_sat()}')

ga_solver = GeneticSolver()
result_ga = ga_solver.solve(p)
print(f'{result_ga.code} {result_ga.get_num_sat()}')

(True, True, True, False, False, False, False, False, True, True) 30
generation[0] :  33
generation[1] :  33
generation[2] :  33
generation[3] :  33
generation[4] :  33
generation[5] :  33
generation[6] :  32
generation[7] :  32
generation[8] :  32
generation[9] :  30
generation[10] :  30
generation[11] :  30
generation[12] :  30
generation[13] :  30
generation[14] :  30
generation[15] :  30
generation[16] :  30
generation[17] :  30
generation[18] :  30
generation[19] :  30
generation[20] :  30
generation[21] :  30
generation[22] :  30
generation[23] :  30
generation[24] :  30
generation[25] :  30
generation[26] :  30
generation[27] :  30
generation[28] :  30
generation[29] :  30
generation[30] :  30
generation[31] :  30
generation[32] :  30
generation[33] :  30
generation[34] :  30
generation[35] :  30
generation[36] :  30
generation[37] :  30
generation[38] :  30
generation[39] :  30
generation[40] :  30
generation[41] :  30
generation[42] :  30
generation[43] :  30
generation[44] : 